In [0]:
%matplotlib inline

In [2]:
from google.colab import files
upload = files.upload()
upload = files.upload()

Saving sudoku.png to sudoku.png


Saving white50.png to white50.png


In [3]:
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pyocr
!curl -L https://github.com/tesseract-ocr/tessdata/raw/master/jpn.traineddata > jpn.traineddata
!cp jpn.traineddata /usr/share/tesseract-ocr/tessdata

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fontconfig libcairo2 libdatrie1 libgif7 libjbig0 liblept5 libopenjp2-7
  libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpixman-1-0
  libtesseract4 libthai-data libthai0 libtiff5 libwebp6 libxcb-render0
  libxcb-shm0 tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  fontconfig libcairo2 libdatrie1 libgif7 libjbig0 liblept5 libopenjp2-7
  libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpixman-1-0
  libtesseract4 libthai-data libthai0 libtiff5 libwebp6 libxcb-render0
  libxcb-shm0 tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 21 newly installed, 0 to remove and 5 not upgraded.
Need to get 8,799 kB of archives.
After this operation, 27.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 fontconfig amd64 2.12.6-0ubuntu2 [169 kB]

In [9]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import sys
import pyocr
import pyocr.builders
from PIL import Image, ImageDraw, ImageFont

#ここから下はOCRが使えるかの確認なのだが、これらがないとtoolが機能しなくなるのでもし完成させるとしたらprintを外して対応

#OCRが使用可能かをチェック
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("OCR tool is not found")
    sys.exit(1)

#OCRツール名を表示
tool = tools[0]
print("OCR tool is '%s'" % (tool.get_name()))

#OCR対応言語を表示
langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))
lang = langs[0]
print("Will use lang '%s'" % (lang))

OCR tool is 'Tesseract (sh)'
Available languages: eng, osd
Will use lang 'eng'


In [10]:
img = cv2.imread("sudoku.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
im2 = cv2.threshold(gray, 140, 225, cv2.THRESH_BINARY_INV)[1]

cnts = cv2.findContours(im2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]

dot = [0, 0, 0, 0, 0]

for pt in cnts:
  X, Y, W, H = cv2.boundingRect(pt)
  #print(X, Y, W, H)
  cv2.rectangle(img, (X, Y), (X+W, Y+H), (0, 255, 0), 30)
  sqware = (W)*(H)
  if sqware > dot[0]:
    dot.clear()
    dot.extend([sqware, X, Y, W, H])

#枠抽出から外を引くパラメーター
area = 20

imgC = img[dot[2]+area:dot[2]+dot[4]-area, dot[1]+area:dot[1]+dot[3]-area]
imgDATA = cv2.resize(imgC, (900, 900))

sudokudata = []

for y in range(9):
  for x in range(9):
    coordinate = (str(y + 1) + str(x + 1))


    #ここが変数になる
    contoursY1 = y * 100
    contoursY2 = contoursY1 + 100
    contoursX1 = x * 100
    contoursX2 = contoursX1 + 100
    #ここが幅のパラメーター
    contours = 12

    filename = "img" + coordinate


    image = imgDATA[contoursY1 + 5 + contours:contoursY2 + 5 - contours,
                    contoursX1 + 2 + contours:contoursX2 + 2 - contours]


    #二値化
    image = cv2.threshold(image,180,255,cv2.THRESH_BINARY)[1]

    #判定に入る
    checkimg = image[10:40, 10:40]
    checkimg = checkimg.flatten()
    if 0 not in checkimg:
      sudokudata.append(0)
    else:
      #print("number")
      #まだ完成形ではないため、数字をpngで出力して余白を追加させる
      cv2.imwrite("{}.png".format(filename), image)

      newimg = Image.open("{}.png".format(filename))
      imgwhite = Image.open('white50.png')
      imgwhite.paste(newimg, (25, 25))
      imgwhite.save("{}a.png".format(filename), quality=95)

      txt = tool.image_to_string(
          Image.open("{}a.png".format(filename)),
          lang="eng",
          builder=pyocr.builders.TextBuilder(tesseract_layout=6))

      #print("{}.png:{}".format(filename, txt))
      sudokudata.append(int(txt))
#Pythonで数独

def encode(grid):
    "テキストから２次元配列のvaluesを作成する"
    values = []
    digits = "123456789"
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    grid_int = map(lambda x: int(x) if x != "." else 0, chars)

    count = 0
    row = []
    for i in grid_int:
        row.append(i)
        count += 1
        if count % 9 == 0: #行毎に分割
            values.append(row)
            row = []
    return values

def main(data):
  if solve(0, 0, data):
    #printtable(data)
    return data
  else:
    print ("This problem is unsolvable")

def solve(x, y, table):
  if x == 0 and y == 9:
    return True

  if table[y][x] == 0:
    for t in range(1, 10):
      table[y][x] = t
      if check(x, y, table):
        (nx, ny) = next(x, y)
        if solve(nx, ny, table):
          return True
    table[y][x] = 0
    return False
  else:
    (nx, ny) = next(x, y)
    if solve(nx, ny, table):
      return True

def next(x, y):
  x += 1
  if x > 8:
    x = 0
    y += 1
  return (x, y)

def check(x, y, table):
  return row(x, y, table) and column(x, y, table) and block(x, y, table)

def column(x, y, table):
  for yt in range(9):
    if yt != y:
      if table[y][x] == table[yt][x]:
        return False
  return True

def row(x, y, table):
  for xt in range(9):
    if xt != x:
      if table[y][x] == table[y][xt]:
        return False
  return True

def block(x, y, table):
  xbase = (x // 3) * 3
  ybase = (y // 3) * 3
  for yt in range(ybase, ybase + 3):
    for xt in range(xbase, xbase + 3):
      if xt != x and yt != y:
        if table[y][x] == table[yt][xt]:
          return False
  return True

def printtable(table):
  for y in range(9):
    for x in range(9):
      print (table[y][x])
    print ("")

def output(data):
    for i in range(9):
        for j in range (9):
            if data[i][j] == 0:
                print (".", end="")
            else:
                print (data[i][j], end="")

            if (j+1) % 3 == 0 and j < 8:
                print ("|", end="")
        print("")
        if (i+1) % 3 == 0 and i < 8:
            print ("-------------------")
    print("")


values = encode(str(sudokudata))
print("This is the problem!")
output(values)
main(values)
print("This is the answer!")
output(values)

print("sucess!")

This is the problem!
..3|2..|8..
.2.|.19|..7
9..|...|.2.
-------------------
..5|1..|6..
.9.|.65|..3
..7|9..|4..
-------------------
..8|5..|9..
5..|...|.6.
.7.|.93|..1

This is the answer!
713|254|896
824|619|357
956|387|124
-------------------
385|142|679
491|765|283
267|938|415
-------------------
148|576|932
539|421|768
672|893|541

sucess!
